<a href="https://colab.research.google.com/github/NonaHelmi/SC/blob/main/antcolony_videoGames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:

import numpy as np

def initialize_pheromone(grid):
    return np.ones(grid.shape)  # Initial pheromone levels

def get_valid_neighbors(pos, grid):
    neighbors = []
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # Up, Down, Left, Right
        new_pos = (pos[0] + dx, pos[1] + dy)
        if (0 <= new_pos[0] < grid.shape[0] and
            0 <= new_pos[1] < grid.shape[1] and
            grid[new_pos] == 0):  # Check if the cell is walkable
            neighbors.append(new_pos)
    return neighbors

def aco_pathfinding(start, goal, grid, num_ants=10, num_iterations=100, alpha=1, beta=2):
    pheromone = initialize_pheromone(grid)
    best_path = None
    best_length = float('inf')

    for iteration in range(num_iterations):
        paths = []
        lengths = []

        for ant in range(num_ants):
            current_pos = start
            path = [current_pos]

            while current_pos != goal:
                valid_neighbors = get_valid_neighbors(current_pos, grid)
                if not valid_neighbors:
                    break  # No path found

                # Calculate probabilities for valid neighbors
                probabilities = []
                for neighbor in valid_neighbors:
                    pheromone_value = pheromone[neighbor]
                    distance = np.linalg.norm(np.array(neighbor) - np.array(goal))
                    heuristic_value = 1 / (distance + 1e-10)  # Avoid division by zero
                    probabilities.append((pheromone_value ** alpha) * (heuristic_value ** beta))

                probabilities = probabilities / np.sum(probabilities)  # Normalize

                next_pos = np.random.choice(range(len(valid_neighbors)), p=probabilities)
                current_pos = valid_neighbors[next_pos]
                path.append(current_pos)

            path_length = len(path)  # Path length is simply the number of steps taken
            paths.append(path)
            lengths.append(path_length)

            # Update pheromone levels based on path length
            for pos in path:
                pheromone[pos] += 1 / path_length

        # Evaporation of pheromone
        pheromone *= 0.95

        # Update best path found
        min_length_idx = np.argmin(lengths)
        if lengths[min_length_idx] < best_length:
            best_length = lengths[min_length_idx]
            best_path = paths[min_length_idx]

    return best_path, best_length

# Example usage for Pathfinding
grid = np.array([[0, 0, 0, 0, 0],
                 [0, 1, 1, 1, 0],
                 [0, 0, 0, 1, 0],
                 [0, 1, 0, 0, 0],
                 [0, 0, 0, 1, 0]])

start = (0, 0)
goal = (4, 4)
best_path, best_length = aco_pathfinding(start, goal, grid)
print("Best path:", best_path)
print("Best path length:", best_length)

Best path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
Best path length: 9
